In [1]:
import sys
import re
import os
import glob
import pandas as pd
import numpy as np

## 1. Import Data

In [88]:
# Import csv file (patent 1948-2014) into Pandas dataframe
# Not including patent issue date or patent number, as claims text only has app_number, cannot match with patent number
# appl_status_date, patent_issue_date may be added back at later time. PEND especially depends on this
info_df = pd.read_csv('../data/application_data.csv')
info_df = info_df[['application_number','filing_date','examiner_id','uspc_class',
                   'uspc_subclass','customer_number','disposal_type']]
print('raw data size:',len(info_df))
info_df.head()

/var/folders/fq/9fcs6ls16fvf714ckvbk_y940000gn/T/ipykernel_90408/4020696485.py:4: DtypeWarning: Columns (0,8,9,10,17,18,19,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  info_df = pd.read_csv('../data/application_data.csv')


raw data size: 9231170


,application_number,filing_date,examiner_id,uspc_class,uspc_subclass,customer_number,disposal_type
0,2000161,NaN,.,NaN,NaN,NaN,PEND
1,2022946,NaN,.,NaN,NaN,NaN,PEND
2,2032617,NaN,.,NaN,NaN,NaN,PEND
3,2045760,1948-08-23,.,NaN,NaN,NaN,ISS
4,2048602,NaN,.,NaN,NaN,NaN,ISS


## 2. Pre-process Data

In [89]:
# Drop NAN examples and examples with a "." which indicate missing data
info_df = info_df[info_df != '.'].dropna()

#explicitly typecast the information columns for easy operations
type_dict = {
    'application_number':'int64',
    'filing_date':'datetime64',
    'examiner_id':'int64',
    'uspc_class':'string',
    'uspc_subclass':'string',
    'customer_number':'int64',
    'disposal_type':'string'
}


# Type casting multiple columns
info_df = info_df.astype(type_dict)

# order by filing date
info_df = info_df.sort_values(by='filing_date')

# after ordering, drop filing date
info_df = info_df.drop('filing_date', axis=1)

# reset index after ordering
info_df = info_df.reset_index(drop=True)


# correct issue of decimal strings being created after uspc class and subclass
def convert_decimal_string(s):
    if isinstance(s, str) and '.' in s:
        try:
            float_val = float(s)
            if float_val.is_integer():
                return str(int(float_val))
        except ValueError:
            pass
    return s

info_df = info_df.applymap(convert_decimal_string)

# check datatype cast
for column in info_df:
    column_type = info_df[column].dtype
    print('column name:',column,', column type:', column_type)

print('Length of data after processing:', len(info_df))
info_df.to_csv('../data/post_application_data.csv', index=False)

# save the post-process data
info_df.to_csv('../data/post_application_data.csv', index=False)

column name: application_number , column type: int64
column name: examiner_id , column type: int64
column name: uspc_class , column type: object
column name: uspc_subclass , column type: object
column name: customer_number , column type: int64
column name: disposal_type , column type: object
Length of data after processing: 4916080


,application_number,examiner_id,uspc_class,uspc_subclass,customer_number,disposal_type
4916030,90013402,97502,362,084000,21917,PEND
4916031,14380549,93008,501,097300,24504,PEND
4916032,14407986,67083,707,755000,27572,PEND
4916033,14571206,67211,709,218000,109883,PEND
4916034,14408036,61369,715,835000,78569,PEND
4916035,14408066,96832,514,006300,23650,PEND
4916036,14408076,61369,715,759000,125706,PEND
4916037,14408086,68118,514,772600,23850,PEND
4916038,14408221,99107,257,098000,27799,PEND
4916039,14571216,67211,709,218000,109883,PEND


## 3. Generate metadata feature set

In [109]:
# create a new, empty dataframe to store the following metadata features (keep it separate from post process)
meta_df = pd.DataFrame(columns=['doc_id','examiner_experience','examiner_allowance_ratio','class_saturation',
                                'subclass_saturation','customer_experience','customer_success_ratio','label'])

# store document id for matching files
meta_df['doc_id'] = info_df['application_number']

# Feature 1,2 examiner experience and allowance rate: # of times examiner_id appeared prior this example
examiner_id_counter = {}
examiner_allowance_counter = {}

def prior_examiner_counts(info_df_row):
    this_examiner = info_df_row['examiner_id']
    
    # feature calculation, based on prior examples
    prior_count = examiner_id_counter.get(this_examiner,0)
    prior_allowed = examiner_allowance_counter.get(this_examiner,0)    
    prior_ratio = prior_allowed / float(prior_count) if prior_count != 0 else 0
    assert prior_count >= 0, 'prior count must be greater or equal to 0'
    assert prior_ratio <= 1.0, 'ratio must be less than 1'
    
    # update counts depending on outcome
    examiner_id_counter[this_examiner] = prior_count + 1
    if info_df_row['disposal_type'] == 'ISS': examiner_allowance_counter[this_examiner] = prior_allowed + 1

    return (prior_count,prior_ratio)
    
results = info_df.apply(prior_examiner_counts, axis=1)
meta_df['examiner_experience'], meta_df['examiner_allowance_ratio'] = zip(*results)
    
# Feature 3, class saturation: # of prior patents in same uspc_class
class_counter = {}
def prior_class_counts(info_df_row):
    this_class = info_df_row['uspc_class']
    prior_count = class_counter.get(this_class,0)
    assert prior_count >= 0, 'prior count must be greater or equal to 0'
    
    class_counter[this_class] = prior_count + 1
    return prior_count

meta_df['class_saturation'] = info_df.apply(prior_class_counts,axis=1)

# Feature 4, subclass saturation: # of prior patents in same uspc_subclass
subclass_counter = {}
def prior_subclass_counts(info_df_row):
    this_subclass = info_df_row['uspc_subclass']
    prior_count = subclass_counter.get(this_subclass,0)
    assert prior_count >= 0, 'prior count must be greater or equal to 0'
    
    subclass_counter[this_subclass] = prior_count + 1
    return prior_count

meta_df['subclass_saturation'] = info_df.apply(prior_subclass_counts,axis=1)

# Feature 5, 6 customer experience and success rate: # of prior applications submitted by same customer_number
customer_id_counter = {}
customer_patent_counter = {}

def prior_customer_counts(info_df_row):
    this_customer = info_df_row['examiner_id']
    
    # feature calculation, based on prior examples
    prior_count = customer_id_counter.get(this_customer,0)
    prior_patents = customer_patent_counter.get(this_customer,0)
    prior_ratio = prior_patents / float(prior_count) if prior_count !=0 else 0
    assert prior_count >= 0, 'prior count must be greater or equal to 0'
    assert prior_ratio <= 1.0, 'ratio must be less than 1'
    
    # update counts depending on outcome
    customer_id_counter[this_customer] = prior_count + 1
    if info_df_row['disposal_type']=='ISS': customer_patent_counter[this_customer] = prior_patents + 1
        
    return (prior_count, prior_ratio)

results = info_df.apply(prior_customer_counts, axis=1)
meta_df['customer_experience'], meta_df['customer_success_ratio'] = zip(*results)

# Feature 7, label: ABN, ISS, PEND
meta_df['label'] = info_df['disposal_type'] 

# save the post-process data
meta_df.to_csv('../data/application_metadata.csv', index=False)
print('metadata dataframe length:',len(meta_df))
print(meta_df.head(10))
print(meta_df.tail(10))

,doc_id,examiner_experience,examiner_allowance_ratio,class_saturation,subclass_saturation,customer_experience,customer_success_ratio,label
0,2682096,0,0.0,0,0,0,0.0,ISS
1,2618355,0,0.0,0,0,0,0.0,ISS
2,3011650,0,0.0,0,0,0,0.0,ISS
3,3237574,0,0.0,0,0,0,0.0,ISS
4,3429712,0,0.0,1,0,0,0.0,ISS
5,4255916,0,0.0,0,0,0,0.0,ISS
6,4458971,0,0.0,0,0,0,0.0,ISS
7,4491494,0,0.0,0,0,0,0.0,ISS
8,4490779,1,1.0,1,1,1,1.0,ISS
9,4510158,0,0.0,0,0,0,0.0,ISS
